In [1]:
%load_ext literary.module

# Import Hook

In [1]:
from functools import lru_cache
from pathlib import Path

from nbconvert import Exporter
from traitlets import Instance, Type, default
from traitlets.config import Configurable

from ..config import find_literary_config, load_literary_config
from ..transpile.exporter import LiteraryExporter
from .loader import NotebookLoader

Here we implement a `ProjectImporter` class. We expose the exporter class to later configuration

In [2]:
class NotebookImporter(Configurable):
    exporter = Instance(Exporter)
    exporter_class = Type(
        LiteraryExporter, help="exporter class used for module source generation"
    ).tag(config=True)

By default, we'll create an instance of `exporter_class`

In [3]:
@patch(NotebookImporter)
@default("exporter")
def _exporter_default(self):
    return self.exporter_class(parent=self)

We will implement a method to build a `NotebookLoader` object using the current exporter

In [4]:
@patch(NotebookImporter)
def get_loader(self, fullname, path):
    # Create the exporter
    exporter = self.exporter_class(parent=self)
    return NotebookLoader(fullname, path, exporter=self.exporter)

## Loader factory
To avoid a startup penalty, we want this module to be imported lazily. However, we also do not want to have to lookup relative imports once the import hook has been installed. It this were to happen, the import hook itself may start resolving against the local notebooks (and things would break)! So, we provide a factory function here that can be imported once on demand by the hook.

In [5]:
def get_loader(fullname, path):
    try:
        config_path = find_literary_config(Path(path))
    except FileNotFoundError:
        return None
    importer = NotebookImporter(config=load_literary_config(config_path))
    return importer.get_loader(fullname, path)